In [1]:
%matplotlib notebook
import keras as keras
from keras.models import Sequential
from keras.layers import LSTM, Dense,BatchNormalization,Dropout,Flatten, Conv1D
from keras.losses import binary_crossentropy, categorical_crossentropy
from keras.metrics import categorical_accuracy
from keras import regularizers,optimizers
from keras.regularizers import l2
import numpy as np
import scipy.io
import matplotlib.pyplot as plt
import gzip
from keras.layers.core import Lambda
from scipy.integrate import trapz
import seaborn as sns

from keras.layers.core import Lambda
from keras import backend as K    


def mini_batches(InputSample,BatchSize):
    Index = np.array(range(InputSample.shape[0]),dtype=int)
    NumBatches = np.int(InputSample.shape[0]/BatchSize)
    Removed = np.array([],dtype=int)

    BatchInd =[]
    for BatchLoop in range(NumBatches):
        RemainIndex = np.delete(Index,Removed)
        SampleInd = np.random.choice(RemainIndex,size=BatchSize,replace=False)
        Removed = np.append(Removed,SampleInd,axis=0)

        BatchInd.append(SampleInd)
    RemainIndex = np.delete(Index,Removed)
    BatchInd.append(RemainIndex)

    return BatchInd,NumBatches
    
def Get_Feats_and_Targets(filename):
    import numpy as np
    
    def line_to_Feats(line):
        line = line.split(' ')
        Feats = np.asarray(line[0:1024])
        Target = np.zeros([3])
        Target[int(line[1024])] = 1
        return Feats,Target

    f = open(filename, 'r')
    lines = f.readlines()
    Features = []
    Targets = []
    for i in range(len(lines)-1):
        line = lines[i+1]
        Feats,Tgts = line_to_Feats(line)
        Features.append(Feats)
        Targets.append(Tgts)
        
    return np.asarray(Features,dtype = 'float64'), np.asarray(Targets,dtype = 'int')

Using TensorFlow backend.


In [2]:
[TrainInputs_Easy,TrainTargets_Easy] = Get_Feats_and_Targets('shapeset1_1cspo_2_3.10000.train.amat')
# [ValInputs_Easy,ValTargets_Easy] = Get_Feats_and_Targets('shapeset1_1cspo_2_3.5000.valid.amat')

[TrainInputs,TrainTargets] = Get_Feats_and_Targets('shapeset2_1cspo_2_3.10000.train.amat')
[ValInputs,ValTargets] = Get_Feats_and_Targets('shapeset2_1cspo_2_3.5000.valid.amat')


In [3]:
TrainInputs = np.append(TrainInputs,TrainInputs_Easy,axis=0)
TrainTargets = np.append(TrainTargets,TrainTargets_Easy,axis=0)

In [4]:
data_dim = TrainInputs.shape[-1]
NumSamples = TrainInputs.shape[0]
Num_Targets = TrainTargets.shape[-1]

index = np.linspace(0,NumSamples,NumSamples,endpoint=False,dtype=int)

reg_coeff = 0

def Gen_Model(reg_coeff):
    model = Sequential()
    model.add(Dense(300,activation='tanh',input_shape =(data_dim,),kernel_regularizer=l2(reg_coeff)))
    model.add(Dropout(0.25))
    model.add(Dense(300,activation='tanh',input_shape =(data_dim,),kernel_regularizer=l2(reg_coeff)))
    model.add(Dropout(0.25))
    model.add(Dense(300,activation='tanh',input_shape =(data_dim,),kernel_regularizer=l2(reg_coeff)))
    model.add(Dropout(0.25))
    model.add(Dense(Num_Targets,activation = 'softmax',kernel_regularizer=l2(reg_coeff),input_shape =(data_dim,)))
    optim = optimizers.adam(lr=0.0001)
    model.compile(loss='categorical_crossentropy',optimizer=optim,metrics=['categorical_accuracy'])
    return model
    
D2THard_model = Gen_Model(reg_coeff)
D2TEasy_model = Gen_Model(reg_coeff)
BALDHard_model = Gen_Model(reg_coeff)
BALDEasy_model = Gen_Model(reg_coeff)

Uni_model = Gen_Model(reg_coeff)

D2THard_model.set_weights(Uni_model.get_weights())
D2TEasy_model.set_weights(Uni_model.get_weights())
BALDHard_model.set_weights(Uni_model.get_weights())
BALDEasy_model.set_weights(Uni_model.get_weights())

In [5]:
def acquisition_function_BALD(model,samples,Num_Targets,temperature=1,Target_Ratio = 5):
    nb_MC_samples = 30
    MC_output = K.function([model.layers[0].input, K.learning_phase()], [model.layers[-1].output])
    MC_samples = np.zeros([nb_MC_samples,samples.shape[0],Num_Targets])
    learning_phase = True 
    for i in range(nb_MC_samples):
        MC_samples[i,:,:] = np.array([MC_output([samples, learning_phase])[0]])
        
    expected_entropy = - np.mean(np.sum(MC_samples * np.log(MC_samples + 1e-10), axis=-1), axis=0)  # [batch size]
    expected_p = np.mean(MC_samples, axis=0)
    entropy_expected_p = - np.sum(expected_p * np.log(expected_p + 1e-10), axis=-1)  # [batch size]
    BALD_acq = entropy_expected_p - expected_entropy
    
    Exp_BALD = np.exp(BALD_acq/temperature)
    Sampling_Prob = Exp_BALD/np.sum(Exp_BALD).astype(float)
    
    Max_Prob_Ratio = Sampling_Prob.max()/Sampling_Prob.min()
    Target_Ratio = Target_Ratio
    if Max_Prob_Ratio < Target_Ratio:
        while Max_Prob_Ratio <Target_Ratio:
            temperature = temperature*0.99
            Exp_BALD = np.exp(BALD_acq/temperature)
            StoreSampling_Prob = Sampling_Prob.copy()
            Sampling_Prob = Exp_BALD/np.sum(Exp_BALD).astype(float)
            Max_Prob_Ratio = Sampling_Prob.max()/Sampling_Prob.min()
            if np.isnan(Max_Prob_Ratio):
                Sampling_Prob = StoreSampling_Prob.copy()
    else:
        while Max_Prob_Ratio > Target_Ratio:
            temperature = temperature*1.01
            Exp_BALD = np.exp(BALD_acq/temperature)
            StoreSampling_Prob = Sampling_Prob.copy()
            Sampling_Prob = Exp_BALD/np.sum(Exp_BALD).astype(float)
            Max_Prob_Ratio = Sampling_Prob.max()/Sampling_Prob.min()
            if np.isnan(Max_Prob_Ratio):
                Sampling_Prob = StoreSampling_Prob.copy()
    return Sampling_Prob

def acquisition_function_BALD_const_temp(model,samples,Num_Targets,temperature=1):
    nb_MC_samples =30
    MC_output = K.function([model.layers[0].input, K.learning_phase()], [model.layers[-1].output])
    MC_samples = np.zeros([nb_MC_samples,samples.shape[0],Num_Targets])
    learning_phase = True 
    for i in range(nb_MC_samples):
        MC_samples[i,:,:] = np.array([MC_output([samples, learning_phase])[0]])
        
    expected_entropy = - np.mean(np.sum(MC_samples * np.log(MC_samples + 1e-10), axis=-1), axis=0)  # [batch size]
    expected_p = np.mean(MC_samples, axis=0)
    entropy_expected_p = - np.sum(expected_p * np.log(expected_p + 1e-10), axis=-1)  # [batch size]
    BALD_acq = entropy_expected_p - expected_entropy
    
    Exp_BALD = np.exp(BALD_acq/temperature)
    Sampling_Prob = Exp_BALD/np.sum(Exp_BALD).astype(float)
    return Sampling_Prob
    
    

In [6]:
def average_dist_to_threshold(model,samples,Num_Targets):
    Output = model.predict(samples)
    Output -= 1/float(Num_Targets)
    Dist_to_Threshold = np.mean(np.abs(Output),1)
    return Dist_to_Threshold

def acquisition_function_dist_to_threshold(model,samples,Num_Targets,temperature=1,Target_Ratio=5):
    Output = model.predict(samples)
    Output -= 1/float(Num_Targets)
    Dist_to_Threshold = np.sum(np.abs(Output),1)
    Exp_Dist_to_Threshold = np.exp(Dist_to_Threshold/temperature)
    Exp_Dist_to_Threshold *= 1
#     Exp_Dist_to_Threshold = Dist_to_Threshold
    Sampling_Prob = Exp_Dist_to_Threshold/np.sum(Exp_Dist_to_Threshold).astype(float)
    Max_Prob_Ratio = Sampling_Prob.max()/Sampling_Prob.min()
    Target_Ratio = Target_Ratio
    if Max_Prob_Ratio < Target_Ratio:
        while Max_Prob_Ratio <Target_Ratio:
            temperature = temperature*0.99
            Exp_Dist_to_Threshold = np.exp(Dist_to_Threshold/temperature)
            StoreSampling_Prob = Sampling_Prob.copy()
            Sampling_Prob = Exp_Dist_to_Threshold/np.sum(Exp_Dist_to_Threshold).astype(float)
            Max_Prob_Ratio = Sampling_Prob.max()/Sampling_Prob.min()
            if np.isnan(Max_Prob_Ratio):
                Sampling_Prob = StoreSampling_Prob.copy()
    else:
        while Max_Prob_Ratio > Target_Ratio:
            temperature = temperature*1.01
            Exp_Dist_to_Threshold = np.exp(Dist_to_Threshold/temperature)
            StoreSampling_Prob = Sampling_Prob.copy()
            Sampling_Prob = Exp_Dist_to_Threshold/np.sum(Exp_Dist_to_Threshold).astype(float)
            Max_Prob_Ratio = Sampling_Prob.max()/Sampling_Prob.min()
            if np.isnan(Max_Prob_Ratio):
                Sampling_Prob = StoreSampling_Prob.copy()
    return Sampling_Prob

def acquisition_function_dist_to_threshold_const_temp(model,samples,Num_Targets,temperature=1):
    Output = model.predict(samples)
    Output -= 1/float(Num_Targets)
    Dist_to_Threshold = np.sum(np.abs(Output),1)
    Exp_Dist_to_Threshold = np.exp(Dist_to_Threshold/temperature)
    Sampling_Prob = Exp_Dist_to_Threshold/np.sum(Exp_Dist_to_Threshold).astype(float)
    return Sampling_Prob

def acquisition_function_entropy(model,samples,Num_Targets,temperature=1,Target_Ratio=5):
    Output = model.predict(samples)
    Entropy = -np.sum(Output * np.log(Output),1)
    Exp_Entropy = np.exp(Entropy/temperature)
    Sampling_Prob = Exp_Entropy/np.sum(Exp_Entropy).astype(float)
    Max_Prob_Ratio = Sampling_Prob.max()/Sampling_Prob.min()
    Target_Ratio = Target_Ratio
    if Max_Prob_Ratio < Target_Ratio:
        while Max_Prob_Ratio <Target_Ratio:
            temperature = temperature*0.99
            Exp_Entropy = np.exp(Entropy/temperature)
            StoreSampling_Prob = Sampling_Prob.copy()
            Sampling_Prob = Exp_Entropy/np.sum(Exp_Entropy).astype(float)
            Max_Prob_Ratio = Sampling_Prob.max()/Sampling_Prob.min()
            if np.isnan(Max_Prob_Ratio):
                Sampling_Prob = StoreSampling_Prob.copy()
    else:
        while Max_Prob_Ratio > Target_Ratio:
            temperature = temperature*1.01
            Exp_Entropy = np.exp(Entropy/temperature)
            StoreSampling_Prob = Sampling_Prob.copy()
            Sampling_Prob = Exp_Entropy/np.sum(Exp_Entropy).astype(float)
            Max_Prob_Ratio = Sampling_Prob.max()/Sampling_Prob.min()
            if np.isnan(Max_Prob_Ratio):
                Sampling_Prob = StoreSampling_Prob.copy()
    return Sampling_Prob

def Exp_ModelChange(Model,Inputs,Num_Targets):
    ExpChange = np.zeros(Inputs.shape[0])
    Model_Output = Model.predict(Inputs)
    for TargetLoop in range(Num_Targets):
        thisTarget = np.zeros([Inputs.shape[0],Num_Targets])
        thisTarget[:,TargetLoop] = 1
        ClassProb = Model_Output[:,TargetLoop]
        Loss = np.mean(Model_Output - Model_Output*thisTarget + np.log(1+np.exp(-Model_Output)),1)
        ExpChange += Loss*ClassProb
    return ExpChange

def acquisition_function_exp_model_change(model,samples,Num_Targets,temperature = 1,Target_Ratio = 5):
    ExpectedChange = Exp_ModelChange(model,samples,Num_Targets)
    ExpChange = np.exp(ExpectedChange/temperature)
    Sampling_Prob = ExpChange/np.sum(ExpChange).astype(float)
    Max_Prob_Ratio = Sampling_Prob.max()/Sampling_Prob.min()
    if Max_Prob_Ratio < Target_Ratio:
        while Max_Prob_Ratio <Target_Ratio:
            temperature = temperature*0.99
            ExpChange = np.exp(ExpectedChange/temperature)
            Sampling_Prob = ExpChange/np.sum(ExpChange).astype(float)
            Max_Prob_Ratio = Sampling_Prob.max()/Sampling_Prob.min()
    else:
        while Max_Prob_Ratio > Target_Ratio:
            temperature = temperature*1.01
            ExpChange = np.exp(ExpectedChange/temperature)
            Sampling_Prob = ExpChange/np.sum(ExpChange).astype(float)
            Max_Prob_Ratio = Sampling_Prob.max()/Sampling_Prob.min()
    return Sampling_Prob

    


In [7]:
def ablate_network(model,ablation_perc):
    weights = model.get_weights()
    save_weights = list(weights)
    NumLayers = len(weights)
    total_numweights  = 0 
    for i in range(NumLayers):
        layer = weights[i]
        total_numweights += len(layer.flatten())
    num_ablations = np.int(ablation_perc*total_numweights)
    weights = ablate_weights(weights,num_ablations)
    model.set_weights(weights)
        
    return(model,save_weights)


def ablation_curve(model,num_tests,max_perc,inputs,targets):
    Perc_Space = np.linspace(0,max_perc,num_tests)
    Performance = np.zeros(num_tests)
    for i in range(num_tests):
        model,save_weights = ablate_network(model,Perc_Space[i])
        Perf = model.test_on_batch(inputs,targets)
        Performance[i] = Perf[1]
        model.set_weights(save_weights)
        AUC = trapz(Performance,Perc_Space)
        
    return AUC,Performance
    
    
def ablate_weights(weights,num_ablations):
    NumLayers = len(weights)
    LayerShape = []
    LayerShape = []
    LayerNodes = [0]
    FlattenedNodes = np.empty([0])
    for i in range(NumLayers):
        LayerShape.append(weights[i].shape)
        LayerNodes.append(len(weights[i].flatten()))
        FlattenedNodes = np.append(FlattenedNodes,weights[i].flatten())
        Index = np.linspace(0,len(FlattenedNodes)-1,num=len(FlattenedNodes),dtype = int)
    RandChoice = np.random.choice(Index,size = num_ablations,replace=False)
    FlattenedNodes[RandChoice] = 0
    FirstInd = 0
    for i in range(NumLayers):
        FirstInd += LayerNodes[i]
        SecondInd = FirstInd + LayerNodes[i+1]
        Sample = FlattenedNodes[FirstInd:SecondInd]
        Sample = Sample.reshape(LayerShape[i])
        weights[i] = Sample
        
    return weights


def acquisition_function_ablation(model,Inputs,Targets,Cluster_Size,temperature=1,Target_Ratio = 5):
    BatchInd,NumBatches = mini_batches(Inputs,Cluster_Size)
    AUC_Record = np.zeros([Inputs.shape[0]])
    count= 0
    for Batch in BatchInd:
        if Batch.shape[0] != 0:
            BatchInputs = Inputs[Batch,:]
            BatchTargets = Targets[Batch,:]
            SaveWeights = model.get_weights()
            model.fit(BatchInputs,BatchTargets,batch_size=64,verbose=0)
            AUC,_ = ablation_curve(model,10,1,Inputs,Targets)
            AUC_Record[Batch] = AUC.copy()
            model.set_weights(SaveWeights)
            count +=1
            
    Exp_AUC = np.exp(AUC_Record/temperature)
    Sampling_Prob = AUC_Record/AUC_Record.sum()
    Max_Prob_Ratio = Sampling_Prob.max()/Sampling_Prob.min()
    if Max_Prob_Ratio < Target_Ratio:
        while Max_Prob_Ratio <Target_Ratio:
            temperature = temperature*0.99
            Exp_AUC = np.exp(AUC_Record/temperature)
            Sampling_Prob = Exp_AUC/np.sum(Exp_AUC).astype(float)
            Max_Prob_Ratio = Sampling_Prob.max()/Sampling_Prob.min()
    else:
        while Max_Prob_Ratio > Target_Ratio:
            temperature = temperature*1.01
            Exp_AUC = np.exp(AUC_Record/temperature)
            Sampling_Prob = Exp_AUC/np.sum(Exp_AUC).astype(float)
            Max_Prob_Ratio = Sampling_Prob.max()/Sampling_Prob.min()

    return Sampling_Prob

In [8]:
count = 0 
Num_Epochs = 256
Num_BurnIn = 1
Batch_Size = 32

NumTasks = 3
try:
    SwitchPoint = np.int(Num_Epochs/NumTasks)
    print(SwitchPoint)
except:
    SwitchPoint = 1
Smoothing_Constant = 0

Val_Error = np.zeros([Num_Epochs,4])
Val_Acc = np.zeros([Num_Epochs,4])



85


In [9]:
Uni_model.fit(TrainInputs,TrainTargets,batch_size = Batch_Size,epochs=Num_Epochs,verbose=1)

Epoch 1/256
20000/20000 [==============================] - 5s 260us/step - loss: 1.1725 - categorical_accuracy: 0.3435
Epoch 2/256
20000/20000 [==============================] - 5s 237us/step - loss: 1.1418 - categorical_accuracy: 0.3549
Epoch 3/256
20000/20000 [==============================] - 5s 240us/step - loss: 1.1289 - categorical_accuracy: 0.3657
Epoch 4/256
20000/20000 [==============================] - 5s 250us/step - loss: 1.1274 - categorical_accuracy: 0.3651
Epoch 5/256
20000/20000 [==============================] - 5s 256us/step - loss: 1.1167 - categorical_accuracy: 0.3767
Epoch 6/256
20000/20000 [==============================] - 5s 242us/step - loss: 1.1093 - categorical_accuracy: 0.3780
Epoch 7/256
20000/20000 [==============================] - 5s 236us/step - loss: 1.1013 - categorical_accuracy: 0.3912
Epoch 8/256
20000/20000 [==============================] - 5s 238us/step - loss: 1.0962 - categorical_accuracy: 0.3928
Epoch 9/256
20000/20000 [=======================

20000/20000 [==============================] - 5s 235us/step - loss: 0.6554 - categorical_accuracy: 0.7154
Epoch 70/256
20000/20000 [==============================] - 5s 235us/step - loss: 0.6558 - categorical_accuracy: 0.7185
Epoch 71/256
20000/20000 [==============================] - 5s 235us/step - loss: 0.6479 - categorical_accuracy: 0.7188
Epoch 72/256
20000/20000 [==============================] - 5s 233us/step - loss: 0.6435 - categorical_accuracy: 0.7180
Epoch 73/256
20000/20000 [==============================] - 5s 235us/step - loss: 0.6493 - categorical_accuracy: 0.7175
Epoch 74/256
20000/20000 [==============================] - 5s 235us/step - loss: 0.6428 - categorical_accuracy: 0.7188
Epoch 75/256
20000/20000 [==============================] - 5s 235us/step - loss: 0.6377 - categorical_accuracy: 0.7211
Epoch 76/256
20000/20000 [==============================] - 5s 234us/step - loss: 0.6281 - categorical_accuracy: 0.7274
Epoch 77/256
20000/20000 [===========================

20000/20000 [==============================] - 5s 238us/step - loss: 0.4848 - categorical_accuracy: 0.7976
Epoch 138/256
20000/20000 [==============================] - 5s 241us/step - loss: 0.4808 - categorical_accuracy: 0.8009
Epoch 139/256
20000/20000 [==============================] - 5s 235us/step - loss: 0.4714 - categorical_accuracy: 0.8038
Epoch 140/256
20000/20000 [==============================] - 5s 236us/step - loss: 0.4690 - categorical_accuracy: 0.8035
Epoch 141/256
20000/20000 [==============================] - 5s 236us/step - loss: 0.4617 - categorical_accuracy: 0.8045
Epoch 142/256
20000/20000 [==============================] - 5s 240us/step - loss: 0.4659 - categorical_accuracy: 0.8057
Epoch 143/256
20000/20000 [==============================] - 5s 233us/step - loss: 0.4658 - categorical_accuracy: 0.8065
Epoch 144/256
20000/20000 [==============================] - 5s 235us/step - loss: 0.4601 - categorical_accuracy: 0.8061
Epoch 145/256
20000/20000 [===================

20000/20000 [==============================] - 5s 234us/step - loss: 0.3860 - categorical_accuracy: 0.8424
Epoch 205/256
20000/20000 [==============================] - 5s 234us/step - loss: 0.3857 - categorical_accuracy: 0.8412
Epoch 206/256
20000/20000 [==============================] - 5s 235us/step - loss: 0.3734 - categorical_accuracy: 0.8499
Epoch 207/256
20000/20000 [==============================] - 5s 232us/step - loss: 0.3787 - categorical_accuracy: 0.8443
Epoch 208/256
20000/20000 [==============================] - 5s 234us/step - loss: 0.3789 - categorical_accuracy: 0.8440
Epoch 209/256
20000/20000 [==============================] - 5s 234us/step - loss: 0.3661 - categorical_accuracy: 0.8516
Epoch 210/256
20000/20000 [==============================] - 5s 235us/step - loss: 0.3779 - categorical_accuracy: 0.8456
Epoch 211/256
20000/20000 [==============================] - 5s 232us/step - loss: 0.3741 - categorical_accuracy: 0.8491
Epoch 212/256
20000/20000 [===================

In [10]:
Dist_to_Threshold = acquisition_function_dist_to_threshold(Uni_model,TrainInputs,Num_Targets,1,10)
BALD = acquisition_function_BALD(Uni_model,TrainInputs,Num_Targets,1,10)

In [11]:
#'Easiest' First
Easy_Dist_to_Threshold_ArgSort = np.flipud(Dist_to_Threshold.argsort())
Hard_Dist_to_Threshold_ArgSort = Dist_to_Threshold.argsort()
Low_BALD_ArgSort = BALD.argsort()
High_BALD_ArgSort = np.flipud(BALD.argsort())

In [ ]:
NumTasks = 5
TaskSplitPoints = np.int(NumSamples/NumTasks)
print(TaskSplitPoints)
for i in range(NumTasks):
#     Num_Epochs_Task = Num_Epochs*(1/(i+1))
    Num_Epochs_Task = Num_Epochs

#     TaskInd = Hard_Dist_to_Threshold_ArgSort[0:(i+1)*TaskSplitPoints]
    TaskInd = Hard_Dist_to_Threshold_ArgSort[(i)*TaskSplitPoints:(i+1)*TaskSplitPoints]
    D2THard_model.fit(TrainInputs[TaskInd,:],TrainTargets[TaskInd],batch_size=Batch_Size,epochs = np.int(Num_Epochs_Task))
    
#     TaskInd = Easy_Dist_to_Threshold_ArgSort[0:(i+1)*TaskSplitPoints]
    TaskInd = Easy_Dist_to_Threshold_ArgSort[(i)*TaskSplitPoints:(i+1)*TaskSplitPoints]
    D2TEasy_model.fit(TrainInputs[TaskInd,:],TrainTargets[TaskInd],batch_size=Batch_Size,epochs = np.int(Num_Epochs_Task))
    
#     TaskInd = Low_BALD_ArgSort[0:(i+1)*TaskSplitPoints]
    TaskInd = Low_BALD_ArgSort[(i)*TaskSplitPoints:(i+1)*TaskSplitPoints]
    BALDEasy_model.fit(TrainInputs[TaskInd,:],TrainTargets[TaskInd],batch_size=Batch_Size,epochs = np.int(Num_Epochs_Task))
    
#     TaskInd = High_BALD_ArgSort[0:(i+1)*TaskSplitPoints]
    TaskInd = High_BALD_ArgSort[(i)*TaskSplitPoints:(i+1)*TaskSplitPoints]
    BALDHard_model.fit(TrainInputs[TaskInd,:],TrainTargets[TaskInd],batch_size=Batch_Size,epochs = np.int(Num_Epochs_Task))
    

4000
Epoch 1/256
4000/4000 [==============================] - 1s 369us/step - loss: 1.1515 - categorical_accuracy: 0.3960
Epoch 2/256
4000/4000 [==============================] - 1s 239us/step - loss: 1.1289 - categorical_accuracy: 0.4103
Epoch 3/256
4000/4000 [==============================] - 1s 235us/step - loss: 1.1119 - categorical_accuracy: 0.4218
Epoch 4/256
4000/4000 [==============================] - 1s 235us/step - loss: 1.1086 - categorical_accuracy: 0.4120
Epoch 5/256
4000/4000 [==============================] - 1s 237us/step - loss: 1.1150 - categorical_accuracy: 0.4123
Epoch 6/256
4000/4000 [==============================] - 1s 236us/step - loss: 1.0996 - categorical_accuracy: 0.4215
Epoch 7/256
4000/4000 [==============================] - 1s 239us/step - loss: 1.0977 - categorical_accuracy: 0.4203
Epoch 8/256
4000/4000 [==============================] - 1s 249us/step - loss: 1.0975 - categorical_accuracy: 0.4220
Epoch 9/256
4000/4000 [==============================] - 1s

4000/4000 [==============================] - 1s 236us/step - loss: 1.0522 - categorical_accuracy: 0.4282
Epoch 71/256
4000/4000 [==============================] - 1s 236us/step - loss: 1.0503 - categorical_accuracy: 0.4405
Epoch 72/256
4000/4000 [==============================] - 1s 238us/step - loss: 1.0490 - categorical_accuracy: 0.4342
Epoch 73/256
4000/4000 [==============================] - 1s 237us/step - loss: 1.0530 - categorical_accuracy: 0.4255
Epoch 74/256
4000/4000 [==============================] - ETA: 0s - loss: 1.0475 - categorical_accuracy: 0.43 - 1s 239us/step - loss: 1.0467 - categorical_accuracy: 0.4390
Epoch 75/256
4000/4000 [==============================] - 1s 243us/step - loss: 1.0496 - categorical_accuracy: 0.4300
Epoch 76/256
4000/4000 [==============================] - 1s 244us/step - loss: 1.0466 - categorical_accuracy: 0.4323
Epoch 77/256
4000/4000 [==============================] - 1s 249us/step - loss: 1.0436 - categorical_accuracy: 0.4368
Epoch 78/256
40

4000/4000 [==============================] - 1s 238us/step - loss: 1.0092 - categorical_accuracy: 0.4723
Epoch 139/256
4000/4000 [==============================] - 1s 238us/step - loss: 1.0097 - categorical_accuracy: 0.4677
Epoch 140/256
4000/4000 [==============================] - 1s 238us/step - loss: 1.0026 - categorical_accuracy: 0.4703
Epoch 141/256
4000/4000 [==============================] - 1s 238us/step - loss: 1.0110 - categorical_accuracy: 0.4587
Epoch 142/256
4000/4000 [==============================] - 1s 239us/step - loss: 1.0091 - categorical_accuracy: 0.4632
Epoch 143/256
4000/4000 [==============================] - 1s 238us/step - loss: 1.0045 - categorical_accuracy: 0.4758
Epoch 144/256
4000/4000 [==============================] - 1s 239us/step - loss: 1.0067 - categorical_accuracy: 0.4652
Epoch 145/256
4000/4000 [==============================] - 1s 238us/step - loss: 1.0039 - categorical_accuracy: 0.4693
Epoch 146/256
4000/4000 [==============================] - 1s 

4000/4000 [==============================] - 1s 238us/step - loss: 0.9509 - categorical_accuracy: 0.5125
Epoch 207/256
4000/4000 [==============================] - 1s 239us/step - loss: 0.9513 - categorical_accuracy: 0.5070
Epoch 208/256
4000/4000 [==============================] - 1s 239us/step - loss: 0.9475 - categorical_accuracy: 0.5210
Epoch 209/256
4000/4000 [==============================] - 1s 238us/step - loss: 0.9442 - categorical_accuracy: 0.5058
Epoch 210/256
4000/4000 [==============================] - 1s 238us/step - loss: 0.9428 - categorical_accuracy: 0.5150
Epoch 211/256
4000/4000 [==============================] - 1s 238us/step - loss: 0.9486 - categorical_accuracy: 0.5067
Epoch 212/256
4000/4000 [==============================] - 1s 238us/step - loss: 0.9408 - categorical_accuracy: 0.5187
Epoch 213/256
4000/4000 [==============================] - 1s 237us/step - loss: 0.9384 - categorical_accuracy: 0.5150
Epoch 214/256
4000/4000 [==============================] - 1s 

4000/4000 [==============================] - 1s 236us/step - loss: 0.4363 - categorical_accuracy: 0.8260
Epoch 19/256
4000/4000 [==============================] - 1s 237us/step - loss: 0.4361 - categorical_accuracy: 0.8285
Epoch 20/256
4000/4000 [==============================] - 1s 247us/step - loss: 0.4126 - categorical_accuracy: 0.8405
Epoch 21/256
4000/4000 [==============================] - 1s 233us/step - loss: 0.4265 - categorical_accuracy: 0.8333
Epoch 22/256
4000/4000 [==============================] - 1s 234us/step - loss: 0.3955 - categorical_accuracy: 0.8455
Epoch 23/256
4000/4000 [==============================] - 1s 235us/step - loss: 0.3772 - categorical_accuracy: 0.8538
Epoch 24/256
4000/4000 [==============================] - 1s 234us/step - loss: 0.3682 - categorical_accuracy: 0.8602
Epoch 25/256
4000/4000 [==============================] - 1s 234us/step - loss: 0.3540 - categorical_accuracy: 0.8645
Epoch 26/256
4000/4000 [==============================] - 1s 234us/st

4000/4000 [==============================] - 1s 234us/step - loss: 0.0591 - categorical_accuracy: 0.9805
Epoch 88/256
4000/4000 [==============================] - 1s 234us/step - loss: 0.0485 - categorical_accuracy: 0.9835
Epoch 89/256
4000/4000 [==============================] - 1s 234us/step - loss: 0.0500 - categorical_accuracy: 0.9843
Epoch 90/256
4000/4000 [==============================] - 1s 234us/step - loss: 0.0422 - categorical_accuracy: 0.9873
Epoch 91/256
4000/4000 [==============================] - 1s 234us/step - loss: 0.0417 - categorical_accuracy: 0.9882
Epoch 92/256
4000/4000 [==============================] - 1s 236us/step - loss: 0.0435 - categorical_accuracy: 0.9870
Epoch 93/256
4000/4000 [==============================] - 1s 237us/step - loss: 0.0333 - categorical_accuracy: 0.9895
Epoch 94/256
4000/4000 [==============================] - 1s 239us/step - loss: 0.0536 - categorical_accuracy: 0.9840
Epoch 95/256
4000/4000 [==============================] - 1s 238us/st

4000/4000 [==============================] - 1s 237us/step - loss: 0.0142 - categorical_accuracy: 0.9960
Epoch 156/256
4000/4000 [==============================] - 1s 238us/step - loss: 0.0200 - categorical_accuracy: 0.9935
Epoch 157/256
4000/4000 [==============================] - 1s 237us/step - loss: 0.0253 - categorical_accuracy: 0.9928
Epoch 158/256
4000/4000 [==============================] - 1s 238us/step - loss: 0.0207 - categorical_accuracy: 0.9928
Epoch 159/256
4000/4000 [==============================] - 1s 236us/step - loss: 0.0296 - categorical_accuracy: 0.9905
Epoch 160/256
4000/4000 [==============================] - 1s 236us/step - loss: 0.0169 - categorical_accuracy: 0.9940
Epoch 161/256
4000/4000 [==============================] - 1s 236us/step - loss: 0.0168 - categorical_accuracy: 0.9938
Epoch 162/256
4000/4000 [==============================] - 1s 237us/step - loss: 0.0285 - categorical_accuracy: 0.9908
Epoch 163/256
4000/4000 [==============================] - 1s 

Epoch 224/256
4000/4000 [==============================] - 1s 235us/step - loss: 0.0070 - categorical_accuracy: 0.9982
Epoch 225/256
4000/4000 [==============================] - 1s 237us/step - loss: 0.0058 - categorical_accuracy: 0.9980
Epoch 226/256
4000/4000 [==============================] - 1s 236us/step - loss: 0.0245 - categorical_accuracy: 0.9933
Epoch 227/256
4000/4000 [==============================] - 1s 237us/step - loss: 0.0300 - categorical_accuracy: 0.9890
Epoch 228/256
4000/4000 [==============================] - 1s 237us/step - loss: 0.0473 - categorical_accuracy: 0.9862
Epoch 229/256
4000/4000 [==============================] - 1s 237us/step - loss: 0.0143 - categorical_accuracy: 0.9948
Epoch 230/256
4000/4000 [==============================] - 1s 237us/step - loss: 0.0072 - categorical_accuracy: 0.9978
Epoch 231/256
4000/4000 [==============================] - 1s 247us/step - loss: 0.0197 - categorical_accuracy: 0.9950
Epoch 232/256
4000/4000 [=======================

4000/4000 [==============================] - 1s 245us/step - loss: 0.3190 - categorical_accuracy: 0.8862
Epoch 37/256
4000/4000 [==============================] - 1s 237us/step - loss: 0.3050 - categorical_accuracy: 0.8898
Epoch 38/256
4000/4000 [==============================] - 1s 238us/step - loss: 0.3026 - categorical_accuracy: 0.8955
Epoch 39/256
4000/4000 [==============================] - 1s 239us/step - loss: 0.2936 - categorical_accuracy: 0.8960
Epoch 40/256
4000/4000 [==============================] - 1s 239us/step - loss: 0.2950 - categorical_accuracy: 0.8942
Epoch 41/256
4000/4000 [==============================] - 1s 238us/step - loss: 0.2973 - categorical_accuracy: 0.8955
Epoch 42/256
4000/4000 [==============================] - 1s 238us/step - loss: 0.2727 - categorical_accuracy: 0.9052
Epoch 43/256
4000/4000 [==============================] - 1s 241us/step - loss: 0.2743 - categorical_accuracy: 0.9000
Epoch 44/256
4000/4000 [==============================] - 1s 241us/st

3616/4000 [==========================>...] - ETA: 0s - loss: 0.0798 - categorical_accuracy: 0.9743

In [16]:
[TestInputs,TestTargets] = Get_Feats_and_Targets('shapeset2_1cspo_2_3.5000.test.amat')

In [24]:
TestError = np.zeros([5,2])
TestError[0,:] = D2THard_model.test_on_batch(TestInputs,TestTargets)
TestError[1,:] = D2TEasy_model.test_on_batch(TestInputs,TestTargets)
TestError[2,:] = BALDHard_model.test_on_batch(TestInputs,TestTargets)
TestError[3,:] = BALDEasy_model.test_on_batch(TestInputs,TestTargets)
TestError[4,:] = Uni_model.test_on_batch(TestInputs,TestTargets)

In [25]:
TestError

array([[0.52482164, 0.7902    ],
       [0.44201103, 0.81840003],
       [0.61129344, 0.75840002],
       [0.52500874, 0.77719998],
       [0.47485489, 0.801     ]])

In [ ]:
D2T Hard  [0.60061926, 0.74940002]
D2T Easy  [0.44051793, 0.81300002]
BALD High [0.60401231, 0.75520003]
BALD Low  [0.45749658, 0.80779999]
Uniform   [0.51151901, 0.79799998]

In [19]:
TestError[1,1] - TestError[-1,1]

0.017400026321411133

In [20]:
Uni_Model_Weights_One_Train = Uni_model.get_weights()

In [21]:
Uni_model.fit(TrainInputs,TrainTargets,batch_size = Batch_Size,epochs=Num_Epochs,verbose=1)

Epoch 1/256
20000/20000 [==============================] - 4s 216us/step - loss: 0.3253 - categorical_accuracy: 0.8710
Epoch 2/256
20000/20000 [==============================] - 4s 213us/step - loss: 0.3292 - categorical_accuracy: 0.8697
Epoch 3/256
20000/20000 [==============================] - 4s 220us/step - loss: 0.3156 - categorical_accuracy: 0.8737
Epoch 4/256
20000/20000 [==============================] - 4s 216us/step - loss: 0.3283 - categorical_accuracy: 0.8688
Epoch 5/256
20000/20000 [==============================] - 4s 217us/step - loss: 0.3262 - categorical_accuracy: 0.8690
Epoch 6/256
20000/20000 [==============================] - 4s 214us/step - loss: 0.3235 - categorical_accuracy: 0.8707
Epoch 7/256
20000/20000 [==============================] - 4s 213us/step - loss: 0.3237 - categorical_accuracy: 0.8707
Epoch 8/256
20000/20000 [==============================] - 4s 215us/step - loss: 0.3205 - categorical_accuracy: 0.8740
Epoch 9/256
20000/20000 [=======================

20000/20000 [==============================] - 4s 214us/step - loss: 0.2906 - categorical_accuracy: 0.8854
Epoch 70/256
20000/20000 [==============================] - 4s 212us/step - loss: 0.2860 - categorical_accuracy: 0.8855
Epoch 71/256
20000/20000 [==============================] - 4s 212us/step - loss: 0.2815 - categorical_accuracy: 0.8887
Epoch 72/256
20000/20000 [==============================] - 4s 213us/step - loss: 0.2718 - categorical_accuracy: 0.8915
Epoch 73/256
20000/20000 [==============================] - 4s 212us/step - loss: 0.2766 - categorical_accuracy: 0.8916
Epoch 74/256
20000/20000 [==============================] - 4s 212us/step - loss: 0.2727 - categorical_accuracy: 0.8915
Epoch 75/256
20000/20000 [==============================] - 4s 212us/step - loss: 0.2765 - categorical_accuracy: 0.8899
Epoch 76/256
20000/20000 [==============================] - 4s 212us/step - loss: 0.2787 - categorical_accuracy: 0.8921
Epoch 77/256
20000/20000 [===========================

20000/20000 [==============================] - 4s 213us/step - loss: 0.2467 - categorical_accuracy: 0.9028
Epoch 138/256
20000/20000 [==============================] - 4s 213us/step - loss: 0.2480 - categorical_accuracy: 0.9005
Epoch 139/256
20000/20000 [==============================] - 4s 213us/step - loss: 0.2512 - categorical_accuracy: 0.9020
Epoch 140/256
20000/20000 [==============================] - 4s 213us/step - loss: 0.2494 - categorical_accuracy: 0.9018
Epoch 141/256
20000/20000 [==============================] - 4s 217us/step - loss: 0.2434 - categorical_accuracy: 0.9045
Epoch 142/256
20000/20000 [==============================] - 4s 216us/step - loss: 0.2470 - categorical_accuracy: 0.9036
Epoch 143/256
20000/20000 [==============================] - 4s 216us/step - loss: 0.2434 - categorical_accuracy: 0.9046
Epoch 144/256
20000/20000 [==============================] - 4s 215us/step - loss: 0.2444 - categorical_accuracy: 0.9056
Epoch 145/256
20000/20000 [===================

20000/20000 [==============================] - 4s 224us/step - loss: 0.2313 - categorical_accuracy: 0.9107
Epoch 205/256
20000/20000 [==============================] - 4s 224us/step - loss: 0.2266 - categorical_accuracy: 0.9103
Epoch 206/256
20000/20000 [==============================] - 4s 217us/step - loss: 0.2274 - categorical_accuracy: 0.9128
Epoch 207/256
20000/20000 [==============================] - 4s 213us/step - loss: 0.2309 - categorical_accuracy: 0.9084
Epoch 208/256
20000/20000 [==============================] - 4s 213us/step - loss: 0.2254 - categorical_accuracy: 0.9117
Epoch 209/256
20000/20000 [==============================] - 4s 213us/step - loss: 0.2203 - categorical_accuracy: 0.9144
Epoch 210/256
20000/20000 [==============================] - 4s 223us/step - loss: 0.2245 - categorical_accuracy: 0.9124
Epoch 211/256
20000/20000 [==============================] - 4s 223us/step - loss: 0.2211 - categorical_accuracy: 0.9129
Epoch 212/256
20000/20000 [===================

In [22]:
TestError = np.zeros([5,2])
TestError[0,:] = D2THard_model.test_on_batch(TestInputs,TestTargets)
TestError[1,:] = D2TEasy_model.test_on_batch(TestInputs,TestTargets)
TestError[2,:] = BALDHard_model.test_on_batch(TestInputs,TestTargets)
TestError[3,:] = BALDEasy_model.test_on_batch(TestInputs,TestTargets)
TestError[4,:] = Uni_model.test_on_batch(TestInputs,TestTargets)
TestError

array([[0.52482164, 0.7902    ],
       [0.44201103, 0.81840003],
       [0.61129344, 0.75840002],
       [0.52500874, 0.77719998],
       [0.37135857, 0.85960001]])

In [ ]:
Uni Trained Twice, No Curriculum
[0.52482164, 0.7902    ],
[0.44201103, 0.81840003],
[0.61129344, 0.75840002],
[0.52500874, 0.77719998],
[0.37135857, 0.85960001]

In [23]:
Uni_model.set_weights(Uni_Model_Weights_One_Train)

In [26]:
NumTasks = 5
TaskSplitPoints = np.int(NumSamples/NumTasks)
print(TaskSplitPoints)
for i in range(NumTasks):
    Num_Epochs_Task = Num_Epochs*(1/(i+1))
    TaskInd = Easy_Dist_to_Threshold_ArgSort[0:(i+1)*TaskSplitPoints]
    Uni_model.fit(TrainInputs[TaskInd,:],TrainTargets[TaskInd],batch_size=Batch_Size,epochs = np.int(Num_Epochs_Task))

4000
Epoch 1/256
4000/4000 [==============================] - 1s 227us/step - loss: 0.0071 - categorical_accuracy: 0.9995
Epoch 2/256
4000/4000 [==============================] - 1s 218us/step - loss: 0.0058 - categorical_accuracy: 0.9998
Epoch 3/256
4000/4000 [==============================] - 1s 216us/step - loss: 0.0041 - categorical_accuracy: 1.0000
Epoch 4/256
4000/4000 [==============================] - 1s 215us/step - loss: 0.0046 - categorical_accuracy: 1.0000
Epoch 5/256
4000/4000 [==============================] - 1s 215us/step - loss: 0.0041 - categorical_accuracy: 0.9998
Epoch 6/256
4000/4000 [==============================] - 1s 215us/step - loss: 0.0037 - categorical_accuracy: 1.0000
Epoch 7/256
4000/4000 [==============================] - 1s 215us/step - loss: 0.0046 - categorical_accuracy: 0.9995
Epoch 8/256
4000/4000 [==============================] - 1s 214us/step - loss: 0.0034 - categorical_accuracy: 1.0000
Epoch 9/256
4000/4000 [==============================] - 1s

4000/4000 [==============================] - 1s 216us/step - loss: 5.3160e-04 - categorical_accuracy: 1.0000
Epoch 70/256
4000/4000 [==============================] - 1s 225us/step - loss: 6.6580e-04 - categorical_accuracy: 1.0000
Epoch 71/256
4000/4000 [==============================] - 1s 223us/step - loss: 5.1340e-04 - categorical_accuracy: 1.0000
Epoch 72/256
4000/4000 [==============================] - 1s 223us/step - loss: 4.2641e-04 - categorical_accuracy: 1.0000
Epoch 73/256
4000/4000 [==============================] - 1s 222us/step - loss: 6.4399e-04 - categorical_accuracy: 1.0000
Epoch 74/256
4000/4000 [==============================] - 1s 222us/step - loss: 0.0086 - categorical_accuracy: 0.9978
Epoch 75/256
4000/4000 [==============================] - 1s 224us/step - loss: 0.0055 - categorical_accuracy: 0.9988
Epoch 76/256
4000/4000 [==============================] - 1s 222us/step - loss: 0.0167 - categorical_accuracy: 0.9948
Epoch 77/256
4000/4000 [=========================

4000/4000 [==============================] - 1s 214us/step - loss: 5.8897e-04 - categorical_accuracy: 1.0000
Epoch 138/256
4000/4000 [==============================] - 1s 215us/step - loss: 2.4742e-04 - categorical_accuracy: 1.0000
Epoch 139/256
4000/4000 [==============================] - 1s 216us/step - loss: 6.3103e-04 - categorical_accuracy: 1.0000
Epoch 140/256
4000/4000 [==============================] - 1s 217us/step - loss: 2.8589e-04 - categorical_accuracy: 1.0000
Epoch 141/256
4000/4000 [==============================] - 1s 217us/step - loss: 3.0056e-04 - categorical_accuracy: 1.0000
Epoch 142/256
4000/4000 [==============================] - 1s 216us/step - loss: 0.0264 - categorical_accuracy: 0.9935
Epoch 143/256
4000/4000 [==============================] - 1s 215us/step - loss: 0.0190 - categorical_accuracy: 0.9938
Epoch 144/256
4000/4000 [==============================] - 1s 215us/step - loss: 0.0014 - categorical_accuracy: 0.9998
Epoch 145/256
4000/4000 [=================

4000/4000 [==============================] - 1s 211us/step - loss: 0.0093 - categorical_accuracy: 0.9970
Epoch 205/256
4000/4000 [==============================] - 1s 213us/step - loss: 0.0064 - categorical_accuracy: 0.9978
Epoch 206/256
4000/4000 [==============================] - 1s 213us/step - loss: 0.0010 - categorical_accuracy: 0.9995
Epoch 207/256
4000/4000 [==============================] - 1s 215us/step - loss: 7.0260e-04 - categorical_accuracy: 1.0000
Epoch 208/256
4000/4000 [==============================] - 1s 214us/step - loss: 3.8170e-04 - categorical_accuracy: 1.0000
Epoch 209/256
4000/4000 [==============================] - 1s 214us/step - loss: 3.5088e-04 - categorical_accuracy: 1.0000
Epoch 210/256
4000/4000 [==============================] - 1s 215us/step - loss: 6.2743e-04 - categorical_accuracy: 0.9998
Epoch 211/256
4000/4000 [==============================] - 1s 215us/step - loss: 2.8691e-04 - categorical_accuracy: 1.0000
Epoch 212/256
4000/4000 [=================

8000/8000 [==============================] - 2s 215us/step - loss: 0.0131 - categorical_accuracy: 0.9967
Epoch 17/128
8000/8000 [==============================] - 2s 215us/step - loss: 0.0283 - categorical_accuracy: 0.9902
Epoch 18/128
8000/8000 [==============================] - 2s 215us/step - loss: 0.0207 - categorical_accuracy: 0.9935
Epoch 19/128
8000/8000 [==============================] - 2s 215us/step - loss: 0.0182 - categorical_accuracy: 0.9938
Epoch 20/128
8000/8000 [==============================] - 2s 216us/step - loss: 0.0136 - categorical_accuracy: 0.9962
Epoch 21/128
8000/8000 [==============================] - 2s 215us/step - loss: 0.0223 - categorical_accuracy: 0.9930
Epoch 22/128
8000/8000 [==============================] - 2s 216us/step - loss: 0.0214 - categorical_accuracy: 0.9932
Epoch 23/128
8000/8000 [==============================] - 2s 215us/step - loss: 0.0273 - categorical_accuracy: 0.9918
Epoch 24/128
8000/8000 [==============================] - 2s 216us/st

8000/8000 [==============================] - 2s 215us/step - loss: 0.0123 - categorical_accuracy: 0.9961
Epoch 86/128
8000/8000 [==============================] - 2s 215us/step - loss: 0.0176 - categorical_accuracy: 0.9950
Epoch 87/128
8000/8000 [==============================] - 2s 215us/step - loss: 0.0139 - categorical_accuracy: 0.9960
Epoch 88/128
8000/8000 [==============================] - 2s 215us/step - loss: 0.0108 - categorical_accuracy: 0.9970
Epoch 89/128
8000/8000 [==============================] - 2s 216us/step - loss: 0.0262 - categorical_accuracy: 0.9911
Epoch 90/128
8000/8000 [==============================] - 2s 215us/step - loss: 0.0120 - categorical_accuracy: 0.9965
Epoch 91/128
8000/8000 [==============================] - 2s 216us/step - loss: 0.0143 - categorical_accuracy: 0.9951
Epoch 92/128
8000/8000 [==============================] - 2s 215us/step - loss: 0.0118 - categorical_accuracy: 0.9971
Epoch 93/128
8000/8000 [==============================] - 2s 215us/st

12000/12000 [==============================] - 3s 214us/step - loss: 0.0411 - categorical_accuracy: 0.9868
Epoch 27/85
12000/12000 [==============================] - 3s 214us/step - loss: 0.0483 - categorical_accuracy: 0.9851
Epoch 28/85
12000/12000 [==============================] - 3s 214us/step - loss: 0.0498 - categorical_accuracy: 0.9836
Epoch 29/85
12000/12000 [==============================] - 3s 214us/step - loss: 0.0554 - categorical_accuracy: 0.9819
Epoch 30/85
12000/12000 [==============================] - 3s 214us/step - loss: 0.0471 - categorical_accuracy: 0.9842
Epoch 31/85
12000/12000 [==============================] - 3s 214us/step - loss: 0.0422 - categorical_accuracy: 0.9862
Epoch 32/85
12000/12000 [==============================] - 3s 214us/step - loss: 0.0370 - categorical_accuracy: 0.9882
Epoch 33/85
12000/12000 [==============================] - 3s 213us/step - loss: 0.0424 - categorical_accuracy: 0.9873
Epoch 34/85
12000/12000 [==============================] - 3

16000/16000 [==============================] - 3s 213us/step - loss: 0.1517 - categorical_accuracy: 0.9480
Epoch 11/64
16000/16000 [==============================] - 3s 212us/step - loss: 0.1451 - categorical_accuracy: 0.9503
Epoch 12/64
16000/16000 [==============================] - 3s 213us/step - loss: 0.1405 - categorical_accuracy: 0.9508
Epoch 13/64
16000/16000 [==============================] - 3s 213us/step - loss: 0.1430 - categorical_accuracy: 0.9517
Epoch 14/64
16000/16000 [==============================] - 3s 212us/step - loss: 0.1388 - categorical_accuracy: 0.9509
Epoch 15/64
16000/16000 [==============================] - 3s 213us/step - loss: 0.1316 - categorical_accuracy: 0.9563
Epoch 16/64
16000/16000 [==============================] - 3s 213us/step - loss: 0.1417 - categorical_accuracy: 0.9512
Epoch 17/64
16000/16000 [==============================] - 3s 213us/step - loss: 0.1348 - categorical_accuracy: 0.9520
Epoch 18/64
16000/16000 [==============================] - 3

20000/20000 [==============================] - 4s 213us/step - loss: 0.2979 - categorical_accuracy: 0.8809
Epoch 16/51
20000/20000 [==============================] - 4s 213us/step - loss: 0.2987 - categorical_accuracy: 0.8836
Epoch 17/51
20000/20000 [==============================] - 4s 213us/step - loss: 0.3008 - categorical_accuracy: 0.8785
Epoch 18/51
20000/20000 [==============================] - 4s 213us/step - loss: 0.2930 - categorical_accuracy: 0.8817
Epoch 19/51
20000/20000 [==============================] - 4s 213us/step - loss: 0.2831 - categorical_accuracy: 0.8860
Epoch 20/51
20000/20000 [==============================] - 4s 213us/step - loss: 0.2875 - categorical_accuracy: 0.8851
Epoch 21/51
20000/20000 [==============================] - 4s 215us/step - loss: 0.2895 - categorical_accuracy: 0.8835
Epoch 22/51
20000/20000 [==============================] - 4s 214us/step - loss: 0.2867 - categorical_accuracy: 0.8847
Epoch 23/51
20000/20000 [==============================] - 4

In [27]:
TestError = np.zeros([5,2])
TestError[0,:] = D2THard_model.test_on_batch(TestInputs,TestTargets)
TestError[1,:] = D2TEasy_model.test_on_batch(TestInputs,TestTargets)
TestError[2,:] = BALDHard_model.test_on_batch(TestInputs,TestTargets)
TestError[3,:] = BALDEasy_model.test_on_batch(TestInputs,TestTargets)
TestError[4,:] = Uni_model.test_on_batch(TestInputs,TestTargets)
TestError

array([[0.52482164, 0.7902    ],
       [0.44201103, 0.81840003],
       [0.61129344, 0.75840002],
       [0.52500874, 0.77719998],
       [0.40061012, 0.83880001]])